<a href="https://colab.research.google.com/github/kswalter1128/FIN654/blob/main/ChurchReport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#Import the packages
import pandas as pd
import os
import datetime as dt
from dateutil.relativedelta import *
import calendar
import numpy as np
os.chdir('/content/drive/MyDrive/Church Financial Report/Raw Data')

#Report Periods
If the reporting period needed for the report isn't the end of month prior to the month you are currently in. In between the quotation marks of `manual

In [27]:

manual_date = input("If for any reporting period other than last month, enter a data in YYYY-MM-DD format, else leave blank: ")
if manual_date=="":
  starting_date=dt.date.today()
else:
  starting_date=dt.datetime.strptime(manual_date, "%Y-%m-%d")+relativedelta(months=1)
report_period=starting_date - relativedelta(months=1)
report_period=report_period + relativedelta(day=31)
prior_period = report_period + relativedelta(months=-1)
prior_year= report_period + relativedelta(years=-1)
print("Current Reporting Period:",report_period,"\nPrior Reporting Period:", prior_period, "\nPrior Year Period:",prior_year)

If for any reporting period other than last month, enter a data in YYYY-MM-DD format, else leave blank: 
Current Reporting Period: 2022-03-31 
Prior Reporting Period: 2022-02-28 
Prior Year Period: 2021-03-31


In [28]:
files = os.listdir()
line = []
for f in files:
    with open(f, 'r') as d:
        data = d.readlines()
        for i in data:
            line.append(i)
dic = {}
dic['RawData'] = line
df = pd.DataFrame(dic)

In [30]:
#Clean up the data
print("Starting Length",len(df['RawData']))
df = df[df['RawData'].str.contains('NATIVITY')==False]
df = df[df['RawData'].str.contains('Total for envelope')==False]
df = df[df['RawData'].str.contains('For Period:')==False]
df = df[df['RawData'].str.contains('Total Posted')==False]
df = df[df['RawData'].str.contains('Posted Contributions')==False]
df = df[df['RawData'].str.contains('Env. # Date')==False]
df = df[df['RawData'].str.startswith("\n")==False]
print("Ending Length",len(df['RawData']))

# Get the Envelope Number Column
df['Envelope Number']=df['RawData'].str.slice(0,6)
df['Envelope Number']=df['Envelope Number'].apply(lambda x: x.strip())
df['Envelope Number']=df['Envelope Number'].apply(lambda x: np.nan if x=="" else x)
df['Envelope Number']=df['Envelope Number'].fillna(method='ffill')
df.head()

#Convert Dates
df['Date']=df['RawData'].str.slice(7,17)
df['Date']=df['Date'].apply(lambda x: dt.datetime.strptime(x,"%m/%d/%Y"))

#get the fund number
df['FundNmbr']=df['RawData'].str.slice(22,25)

#get the fund name
df['RawData']=df['RawData'].str.slice(25)
df['Fund Name']=df['RawData'].str.extract("(.+\s)(?:[\d,-]+[.][\d{2}])")
df['Fund Name']=df['Fund Name'].str.strip()

#get the amount
df['Amount']=df['RawData'].str.extract("([-]?(?!,$)[\d,]+[.]\d{2})")
df['Amount']=df['Amount'].apply(lambda x: x.replace(",",""))
df['Amount']=df['Amount'].astype(float)

df.head()

Starting Length 31866
Ending Length 31866


,RawData,Envelope Number,Date,FundNmbr,Fund Name,Amount
5,Memorial - Jeff Hegedus 100.00 ...,9519,2021-09-19,308,Memorial - Jeff Hegedus,100.0
7,General Fund-P 100.00 ...,24,2021-09-05,101,General Fund-P,100.0
8,General Benev.-P 40.00 ...,24,2021-09-05,201,General Benev.-P,40.0
11,General Fund-NP 100.00 ...,403,2021-09-05,102,General Fund-NP,100.0
13,General Fund-P 80.00 ...,291,2021-09-12,101,General Fund-P,80.0


In [31]:
#remove RawData Column
df.drop(columns='RawData', inplace=True)
df.head()

,Envelope Number,Date,FundNmbr,Fund Name,Amount
5,9519,2021-09-19,308,Memorial - Jeff Hegedus,100.0
7,24,2021-09-05,101,General Fund-P,100.0
8,24,2021-09-05,201,General Benev.-P,40.0
11,403,2021-09-05,102,General Fund-NP,100.0
13,291,2021-09-12,101,General Fund-P,80.0


In [32]:
funds = df[['Date','FundNmbr','Fund Name']]
funds.head()

,Date,FundNmbr,Fund Name
5,2021-09-19,308,Memorial - Jeff Hegedus
7,2021-09-05,101,General Fund-P
8,2021-09-05,201,General Benev.-P
11,2021-09-05,102,General Fund-NP
13,2021-09-12,101,General Fund-P


In [33]:
funds.sort_values(by=['FundNmbr', 'Date'], ascending=[True,False], inplace=True)
funds.drop(columns='Date', inplace=True)
funds.drop_duplicates(inplace=True)
funds.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,FundNmbr,Fund Name
9390,101,General Fund-P
9407,102,General Fund-NP
10223,103,Plate
10193,105,SCS loose cash
9906,106,Initial


In [17]:
pd.period_range(start=df['Date'].min(), end=df['Date'].max(), freq='D')

PeriodIndex(['2019-01-06', '2019-01-07', '2019-01-08', '2019-01-09',
             '2019-01-10', '2019-01-11', '2019-01-12', '2019-01-13',
             '2019-01-14', '2019-01-15',
             ...
             '2022-03-18', '2022-03-19', '2022-03-20', '2022-03-21',
             '2022-03-22', '2022-03-23', '2022-03-24', '2022-03-25',
             '2022-03-26', '2022-03-27'],
            dtype='period[D]', length=1177)